In [2]:
import sys
sys.path.append("../../")
from Scripts import *

In [ ]:
results_dict = {}
cost_curve_dicts = {'Optimization': {}, 'Planning': {}}

opt_counter = 0
plan_counter = 0
for n_sats in range(3,13,1):
    results_dict[n_sats] = {'Optimization': {'Total Results': pd.DataFrame({'Number of Targets': [], 'Avg_Percentage': [], 'Max_Time': [], 'Avg_Time': [], 'Std_Time': []})}, 'Planning': pd.DataFrame({'Total Results': {'Number of Targets': [], 'Avg_Percentage': [], 'Max_Time': [], 'Avg_Time': [], 'Std_Time': []}})}


    if os.path.exists(f'../../Pop_Over_Gen/{n_sats}.csv'):
        df = pd.read_csv(f'../../Pop_Over_Gen/{n_sats}.csv')
        results_dict[n_sats]['Pop_Over_Gen'] = df


    for n_targets in [15,34,65,82,109,186,494,1131]:
        if os.path.exists(f'../../Optimization_Results/{n_sats}_{n_targets}.csv'):
            df = pd.read_csv(f'../../Optimization_Results/{n_sats}_{n_targets}.csv').round(4)
            results_dict[n_sats]['Optimization'][n_targets] = df
            results_dict[n_sats]['Optimization']['Total Results'].loc[len(results_dict[n_sats]['Optimization']['Total Results'])] = [df["Unplanned (%)"].count(), df["Unplanned (%)"].mean().round(4), df["Unplanned (Time)"].max().round(1),  df["Unplanned (Time)"].mean().round(1), df["Unplanned (Time)"].std().round(1)]
            opt_counter += 1

        if os.path.exists(f'../../Planning_Results/{n_sats}_{n_targets}.csv'):
            df = pd.read_csv(f'../../Planning_Results/{n_sats}_{n_targets}.csv').round(4)
            results_dict[n_sats]['Planning'][n_targets] = df
            results_dict[n_sats]['Planning']['Total Results'].loc[len(results_dict[n_sats]['Planning']['Total Results'])] = [df["Planned (%)"].count(), df["Planned (%)"].mean().round(4), df["Planned (Time)"].max().round(1), df["Planned (Time)"].mean().round(1), df["Planned (Time)"].std().round(1)]
            plan_counter += 1

    cost_curve_dicts['Optimization'][n_sats] = results_dict[n_sats]['Optimization']['Total Results'][results_dict[n_sats]['Optimization']['Total Results']['Avg_Percentage'] == 100]
    if len(results_dict[n_sats]['Planning']['Total Results']['Number of Targets']) != 0:
        cost_curve_dicts['Planning'][n_sats] = results_dict[n_sats]['Planning']['Total Results'][results_dict[n_sats]['Planning']['Total Results']['Avg_Percentage'] == 100]


    

# axis's = Number of Targets, Avg_Time, Max_Time, Std_Time
# curve_type = Optimzation, Planning
Generate_Performance_Curve(cost_curve_dicts,curve_type='Optimization', xaxis='Number of Targets', yaxis='Max_Time') 
Generate_Performance_Curve(cost_curve_dicts,curve_type='Optimization', xaxis='Number of Targets', yaxis='Avg_Time') 

In [ ]:
total_runs = 80
pd.DataFrame({'Type': ['Optimization', 'Planning'], 'Runs Left': [80 - opt_counter , 80 - plan_counter], '% Done': [f"{(opt_counter/80) * 100:.2f} %",f"{(plan_counter/80) * 100:.2f} %"]})


In [ ]:
def pop_over_gen_plotter_animator(n_sats, results_dict, hover_text=['Alt', 'Inc', 'Initial_Raan', 'Delta_Raan','Num_Planes', 'Avg_Percentage', 'Std_Percentage', 'Avg_Time',
       'Std_Time'],plot_title='Satellites Pop_Over_Gen', xaxis='Avg_Time', yaxis='Avg_Percentage', graph=True, animation=True, constraint=True, xticks = [0, 10, 20, 30, 40], yticks=[0, 25, 50, 75, 100]):

    pop_over_gen_df = results_dict[n_sats]['Pop_Over_Gen']
    colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan', 'olive', 'yellow']

    figs = []
    if graph:
        fig_graph = make_subplots()
        figs.append(fig_graph)
        
    if animation:
        fig_animation = make_subplots()
        figs.append(fig_animation)
    

    frames = []

    df = pop_over_gen_df[pop_over_gen_df['Gen'] == 0]

    scatter = go.Scatter(
        x=df[xaxis],
        y=df[yaxis],
        hovertext=df[hover_text].astype(str).agg(', '.join, axis=1),
        hoverinfo='text',
        mode='markers',
        name=f'{0}',
        marker=dict(color=colors[0], size=8),
    )

    

    for i, gen in enumerate(pop_over_gen_df['Gen'].unique()):
        df = pop_over_gen_df[pop_over_gen_df['Gen'] == gen]


        scatter = go.Scatter(
            x=df[xaxis],
            y=df[yaxis],
            hovertext=df[hover_text].astype(str).agg(', '.join, axis=1),
            hoverinfo='text',
            mode='markers',
            name=f'{gen}',
            marker=dict(color=colors[i % len(colors)], size=8),
        )
        
        if i == 0 and animation:
            fig_animation.add_trace(scatter)

        if graph:
            fig_graph.add_trace(scatter)

        frames.append(go.Frame(data=[scatter], name=str(gen)))

    for fig in figs:

        if constraint:
            fig.add_vline(30, line_dash='dash', line_color='red')
            fig.add_hline(100, line_dash='dash', line_color='red')

            fig.add_annotation(x=30.5, y=50, text='30 Day Constraint', font=dict(color='red', size=15), textangle=90, showarrow=False)
            fig.add_annotation(x=15, y=105, text='100% Imaged Constraint', font=dict(color='red', size=15), showarrow=False)


        if graph and fig == fig_graph:
            fig.update_layout(
                title=f'{plot_title}',
                xaxis_title=xaxis,
                yaxis_title=yaxis,
                xaxis=dict(tickmode='array',tickvals=xticks),
                yaxis=dict(tickmode='array',tickvals=yticks, range=[yticks[0]-10, yticks[-1]+10]),
                legend=dict(title='Gen',title_font=dict(size=15),font=dict(size=12, color='black')),
                font=dict(size=15))

        if animation and fig == fig_animation:
            
            fig.update_layout(title=f'{plot_title}',
            xaxis_title=xaxis,
            yaxis_title=yaxis,
            xaxis=dict(tickmode='array',tickvals=xticks),
            yaxis=dict(tickmode='array',tickvals=yticks,range=[yticks[0]-10, yticks[-1]+10]),
            legend=dict(title='Gen',title_font=dict(size=15),font=dict(size=12, color='black')),
            font=dict(size=15), 
            updatemenus=[{
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=[{
            'active': 0,
            'yanchor': 'top',
            'xanchor': 'left',
            'currentvalue': {
                'font': {'size': 20},
                'prefix': 'Generation: ',
                'visible': True,
                'xanchor': 'right'
            },
            'transition': {'duration': 300, 'easing': 'cubic-in-out'},
            'pad': {'b': 10, 't': 50},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [
                {
                    'args': [[str(gen)], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                    'label': str(gen),
                    'method': 'animate'
                } for gen in pop_over_gen_df['Gen'].unique()
            ]
        }])
            fig.frames = frames
        fig.show()

n_sats=5
pop_over_gen_plotter_animator(n_sats, results_dict, graph=True, animation=True, xaxis='Avg_Time', yaxis='Avg_Percentage',plot_title=f'{n_sats} Satellite Constellation Populations Over Generations')
